In [1]:
import geopandas as gpd
import pandas as pd 

In [2]:
circo_df = gpd.read_file("data/circoscrizioni.json")
circo_df = circo_df[['numero_cir', 'nome', 'area', 'geometry']]
circo_df = circo_df.sort_values("nome", ascending=True)
circo_df = circo_df.reset_index()
circo_area_col = circo_df['area']
circo_area_col

0     13199398
1     36949609
2      8599464
3      8817959
4     16409555
5     15712181
6      6962104
7     15433274
8     16687779
9      3073349
10     8663138
11     7342864
Name: area, dtype: int64

In [3]:
tree_df = gpd.read_file("data/geo_data_trees.geo.json")
tree_df = tree_df[['Tree ID', 'Canopy Cover (m2)', 'Oxygen Production (kg/yr)', 'geometry']]
tree_df = tree_df.rename(columns={"Oxygen Production (kg/yr)": "Oxygen", "Canopy Cover (m2)": "Canopy" })
tree_df = tree_df.astype({"Oxygen":"float", "Canopy":"float" })
tree_df.head()

,Tree ID,Canopy,Oxygen,geometry
0,1.0,1.8,2.8,POINT (11.07201 46.05160)
1,2.0,1.8,3.0,POINT (11.07211 46.05154)
2,3.0,1.8,3.4,POINT (11.07196 46.05167)
3,4.0,4.9,4.3,POINT (11.15017 46.06378)
4,7.0,1.8,29.5,POINT (11.07155 46.05230)


In [4]:
def get_circoscrizione(tree_point):
    for idx_polygon, polygon in circo_geometry_col.items():
        if tree_point == None: return "None"
        if tree_point.within(polygon):
            return  circo_df['nome'][idx_polygon]

circo_geometry_col = circo_df["geometry"]        
tree_df = tree_df.copy()
tree_df['circoscrizione'] = ""
tree_df['circoscrizione']= tree_df['geometry'].map(get_circoscrizione)
tree_df['count'] = tree_df.groupby(['circoscrizione'])['circoscrizione'].transform('count')
tree_df['canopy_tot'] = tree_df.groupby(['circoscrizione'])['Canopy'].transform('sum')
tree_df['oxygen_tot'] = tree_df.groupby(['circoscrizione'])['Oxygen'].transform('sum')

tree_df.head()

,Tree ID,Canopy,Oxygen,geometry,circoscrizione,count,canopy_tot,oxygen_tot
0,1.0,1.8,2.8,POINT (11.07201 46.05160),SARDAGNA,38,389.2,580.5
1,2.0,1.8,3.0,POINT (11.07211 46.05154),SARDAGNA,38,389.2,580.5
2,3.0,1.8,3.4,POINT (11.07196 46.05167),SARDAGNA,38,389.2,580.5
3,4.0,4.9,4.3,POINT (11.15017 46.06378),POVO,189,2866.9,2762.2
4,7.0,1.8,29.5,POINT (11.07155 46.05230),SARDAGNA,38,389.2,580.5


In [5]:
tree_df = tree_df.drop(["Tree ID", "Canopy", "Oxygen", "geometry"], axis=1)
tree_df.head()

,circoscrizione,count,canopy_tot,oxygen_tot
0,SARDAGNA,38,389.2,580.5
1,SARDAGNA,38,389.2,580.5
2,SARDAGNA,38,389.2,580.5
3,POVO,189,2866.9,2762.2
4,SARDAGNA,38,389.2,580.5


In [6]:
tree_df = tree_df.drop_duplicates()
tree_df = tree_df[tree_df.circoscrizione!="None"]
tree_df = tree_df.sort_values(by='circoscrizione', ascending=True)
tree_df = tree_df.reset_index()

In [7]:
tree_df['area'] = circo_area_col

In [8]:
tree_df['density'] = ''
tree_df['density'] = tree_df['canopy_tot'] / tree_df['area']
tree_df

,index,circoscrizione,count,canopy_tot,oxygen_tot,area,density
0,1501,ARGENTARIO,454,12416.1,6848.3,13199398,0.000941
1,40,BONDONE,292,5979.6,4559.9,36949609,0.000162
2,1644,CENTRO STORICO PIEDICASTELLO,2964,131189.1,53818.4,8599464,0.015255
3,477,GARDOLO,1651,27279.5,12353.7,8817959,0.003094
4,116,MATTARELLO,264,4995.4,3492.6,16409555,0.000304
5,1497,MEANO,206,3136.1,2961.1,15712181,0.000200
6,212,OLTREFERSINA,2695,87988.8,43064.0,6962104,0.012638
7,3,POVO,189,2866.9,2762.2,15433274,0.000186
8,20,RAVINA-ROMAGNANO,319,6289.5,3680.8,16687779,0.000377
9,1693,S.GIUSEPPE-S.CHIARA,3024,93333.1,51717.3,3073349,0.030369
